In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from googletrans import Translator
import logging
import time
from deep_translator import GoogleTranslator
# Import the logging module to handle errors
logging.basicConfig(level=logging.ERROR)

In [2]:
!pip install googletrans==3.1.0a0

In [3]:
def translate_to_sinhala(text):
    time.sleep(1)
    translator = Translator()
    time.sleep(1)
    translation = translator.translate(text, src='en', dest='si')
    time.sleep(1)
    return translation.text

In [4]:
# def translate_to_sinhala(text):
#     translation = GoogleTranslator(source='en', target='si').translate(text)
#     return translation

In [5]:
def convert_txt_to_csv(file_path, encoding='utf-16'):
    with open(file_path, 'r', encoding=encoding) as file:
        content = file.read()
    lines = content.strip().split('\n')
    return lines

In [6]:
# convert_txt_to_csv('./data/Singlish/Romanized Sinhala- sinhala cleaned data.txt')

In [7]:
sinhala_to_singlish_dict = {}
singlish_to_sinhala_dict = {}

In [8]:
from tqdm import tqdm

In [9]:
for word_combo in tqdm(convert_txt_to_csv('./data/Singlish/Swa Bhasha D 1.txt', encoding='utf-8')):
    singlish, sinhala = word_combo.split('/')
    sinhala_to_singlish_dict[sinhala.strip().lower()] = singlish.strip().lower()
    singlish_to_sinhala_dict[singlish.strip().lower()] = sinhala.strip().lower()

100%|█████████████████████████████| 7122355/7122355 [00:09<00:00, 789141.70it/s]


In [10]:
for word_combo in tqdm(convert_txt_to_csv('./data/Singlish/test1.txt')):
    sinhala, singlish = word_combo.split(',')
    sinhala_to_singlish_dict[sinhala.strip().lower()] = singlish.strip().lower()
    singlish_to_sinhala_dict[singlish.strip().lower()] = sinhala.strip().lower()

100%|███████████████████████████████| 440024/440024 [00:00<00:00, 640290.48it/s]


In [11]:
# convert_txt_to_csv('./data/Singlish/WSD Romanized-Sinhala - Sinhala .txt', encoding="utf-8")

In [12]:
list(sinhala_to_singlish_dict.values())[:5]

['yogyathama', 'yogyatha', 'yogya', 'yogini', 'yogat']

In [13]:
list(singlish_to_sinhala_dict.values())[:5]

['යෝග්\u200dයතම', 'යෝග්\u200dයතා', 'යොග්\u200dය', 'යෝගිනී', 'යෝගට්']

In [14]:
# english_text = "Hello, how are you?"
# sinhala_text = translate_to_sinhala(english_text)
# print(f"English: {english_text}")
# print(f"Sinhala: {sinhala_text}")

In [15]:
tweet_gpt = pd.read_csv('./data/ChatBot/TweetChatGPT.csv')[['Tweet', 'Description']]

/tmp/ipykernel_82183/1497944970.py:1: DtypeWarning: Columns (0,6,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet_gpt = pd.read_csv('./data/ChatBot/TweetChatGPT.csv')[['Tweet', 'Description']]


In [16]:
general_chatbot = pd.read_csv('./data/ChatBot/GeneralCovos.csv')[['question', 'answer']]

In [17]:
general_chatbot.columns = ['q', 'a']

In [18]:
tweet_gpt.columns = ['q', 'a']

In [19]:
data = pd.concat([general_chatbot, tweet_gpt])

In [20]:
data

,q,a
0,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
1,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
2,You're asking me out. That's so cute. What's ...,Forget it.
3,"No, no, it's my fault -- we didn't have a prop...",Cameron.
4,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.
...,...,...
478342,@OpenAI 's new #ChatGPT is insane\nPrompt: An ...,Creator of TikTokApi Python Package | General ...
478343,ChatGPT first look - https://t.co/16K2NlmOoY. ...,"Build, test, experiment, deploy and enjoy the ..."
478344,OK so @OpenAI's new #ChatGPT can basically jus...,building things with #AI 🤖 #DALLE & #MidJourne...
478345,Just in! \n#ChatGPT - research early stage GPT...,"Co-Founder, CIO & COO @ IDWise - Trust but ver..."


In [21]:
data.iloc[0]

q    Well, I thought we'd start with pronunciation,...
a    Not the hacking and gagging and spitting part....
Name: 0, dtype: object

In [22]:
def sinhala_to_singlish(sentence):
    sentence = remove_other_characters(sentence).split(' ')
    singlish_words = []
    for word in sentence:
        if word in sinhala_to_singlish_dict.keys():
            singlish_words.append(sinhala_to_singlish_dict[word])
        else:
            singlish_words.append("[Unkown]")
    return " ".join(singlish_words)

In [23]:
import re

In [24]:
random_characters = ['!', '"', "'", '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
remove_other_characters = lambda sentence: re.sub(r'\s+', ' ', ''.join(char if char not in random_characters else ' ' for char in sentence))

In [25]:
converted_data_dict = {
    "Sinhala":[],
    "English":[],
    "Singlish":[]
}

In [26]:
import threading
from queue import Queue

In [ ]:
def save_data(converted_data_dict):
    df = pd.DataFrame(converted_data_dict)
    df.to_csv('converted_data.csv', index=False)

def worker():
    while True:
        idx = idx_queue.get()
        if idx is None:
            break
        row = data.iloc[idx]
        english = row['q'] + row['a']
        english = english.strip().lower()
        english = remove_other_characters(english)
        sinhala = translate_to_sinhala(english)
        singlish = sinhala_to_singlish(sinhala)
        with lock:
            converted_data_dict['Sinhala'].append(sinhala)
            converted_data_dict['English'].append(english)
            converted_data_dict['Singlish'].append(singlish)
            # Save data every time it is updated
            save_data(converted_data_dict)
        idx_queue.task_done()
        progress_bar.update(1)

# Number of threads to use
num_threads = 6  # Adjust as needed

# Initialize data structures
converted_data_dict = {'Sinhala': [], 'English': [], 'Singlish': []}
idx_queue = Queue()
lock = threading.Lock()

# Add indices to the queue with progress bar
with tqdm(total=len(data), desc="Queueing", unit="rows") as queueing_bar:
    for idx in range(len(data)):
        idx_queue.put(idx)
        queueing_bar.update(1)

# Start worker threads
threads = []
for _ in range(num_threads):
    thread = threading.Thread(target=worker)
    thread.start()
    threads.append(thread)

# Use tqdm to track progress
with tqdm(total=len(data), desc="Processing", unit="rows") as progress_bar:
    for _ in range(num_threads):
        idx_queue.put(None)

    # Wait for all tasks in the queue to be processed
    idx_queue.join()

# Wait for all threads to finish
for thread in threads:
    thread.join()

Processing:   2%|▎                | 11656/617756 [2:00:13<85:59:40,  1.96rows/s]Exception in thread Thread-7 (worker):
Traceback (most recent call last):
  File "/home/ranuga/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
Exception in thread Thread-9 (worker):
Traceback (most recent call last):
  File "/home/ranuga/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/ranuga/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_82183/1472033634.py", line 14, in worker
    self.run()
  File "/home/ranuga/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_82183/1472033634.py", line 14, in worker
  File "/tmp/ipykernel_82183/766331838.py", line 5, in translate_to_sinhala
  File "/tmp/ipykernel_82183/766331838.py", line 5, in translate_to_sinhala
  File "/home/ranuga/anaconda3